In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Reviews-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://mychallengebucket1819.s3.us-east-2.amazonaws.com/amazon_reviews_us_Tools_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [7]:
# ReCreate the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [8]:
#2.1 Filter the data and create a new DataFrame where the total_votes count is equal to or greater than 20
filtered_df = vine_df.filter("total_votes>=20")
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [9]:
#2.2 create a new DataFrame to retrieve all the rows where the number of helpful_votes
# divided by total_votes is equal to or greater than 50%
new_filtered_df = filtered_df.filter("helpful_votes / total_votes >=0.50")
new_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [10]:
#2.3 create a new DataFrame that retrieves all the rows where a review 
#was written as part of the Vine program (paid), vine == 'Y'
new_y_vine_df = new_filtered_df.filter(new_filtered_df.vine == 'Y')
new_y_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|
|R2QT2G3YRGC0PS|          5|           18|         20|   Y|                N|
| RDUP15OURNUSA|          5|           22|         23|   Y|                N|
|R1EI4NZTG7G14F|          5|           17|         23|   Y|     

In [11]:
#2.4 retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
new_n_vine_df = new_filtered_df.filter(new_filtered_df.vine == 'N')
new_n_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [14]:
#2.5 Determine the total number of reviews, the number of 5-star reviews,
#and the percentage of 5-star reviews for the two types of review (paid vs unpaid)

#total number of reviews
y_vine_total = new_y_vine_df.count()
n_vine_total = new_n_vine_df.count()
total_reviews = y_vine_total + n_vine_total
print("Total number of Vine reviews: %f" % y_vine_total)
print("Total number of non-Vine reviews: %f" % n_vine_total)
print("Total number of reviews: %f" % total_reviews)

#number of 5star reviews
y_vine_5star = new_y_vine_df.filter(new_y_vine_df.star_rating =='5').count()
n_vine_5star = new_n_vine_df.filter(new_n_vine_df.star_rating =='5').count()
total_5star_reviews = y_vine_5star + n_vine_5star
print("Total number of 5 star Vine reviews: %f" % y_vine_5star)
print("Total number of 5 star non-Vine reviews: %f" % n_vine_5star)
print("Total number of 5 star reviews: %f" % total_5star_reviews)

#percent of 5star for paid vs unpaid
paid_reviews = (y_vine_5star / y_vine_total) * 100
unpaid_reviews = (n_vine_5star / n_vine_total) * 100
print("Total percent of Vine Paid reviews: %f" % paid_reviews)
print("Total percent of Vine Unpaid reviews: %f" % unpaid_reviews)

Total number of Vine reviews: 285.000000
Total number of non-Vine reviews: 31545.000000
Total number of reviews: 31830.000000
Total number of 5 star Vine reviews: 163.000000
Total number of 5 star non-Vine reviews: 14614.000000
Total number of 5 star reviews: 14777.000000
Total percent of Vine Paid reviews: 57.192982
Total percent of Vine Unpaid reviews: 46.327469
